## Исследование надёжности заёмщиков

Заказчик — кредитный отдел банка. Нужно разобраться, влияет ли семейное положение и количество детей клиента на факт погашения кредита в срок. Входные данные от банка — статистика о платёжеспособности клиентов.

Результаты исследования будут учтены при построении модели **кредитного скоринга** — специальной системы, которая оценивает способность потенциального заёмщика вернуть кредит банку.

### Шаг 1. Откройте файл с данными и изучите общую информацию. 

In [1]:
import pandas as pd
df = pd.read_csv('data_1_project.csv')
df

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу
...,...,...,...,...,...,...,...,...,...,...,...,...
21520,1,-4529.316663,43,среднее,1,гражданский брак,1,F,компаньон,0,224791.862382,операции с жильем
21521,0,343937.404131,67,среднее,1,женат / замужем,0,F,пенсионер,0,155999.806512,сделка с автомобилем
21522,1,-2113.346888,38,среднее,1,гражданский брак,1,M,сотрудник,1,89672.561153,недвижимость
21523,3,-3112.481705,38,среднее,1,женат / замужем,0,M,сотрудник,1,244093.050500,на покупку своего автомобиля


In [2]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       19351 non-null float64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        19351 non-null float64
purpose             21525 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


В колонках days_employed и total_income имеется порядка 2000 пропущенных значений, взглянем на них:

In [3]:
df[df.days_employed.isna()]

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
12,0,NaN,65,среднее,1,гражданский брак,1,M,пенсионер,0,NaN,сыграть свадьбу
26,0,NaN,41,среднее,1,женат / замужем,0,M,госслужащий,0,NaN,образование
29,0,NaN,63,среднее,1,Не женат / не замужем,4,F,пенсионер,0,NaN,строительство жилой недвижимости
41,0,NaN,50,среднее,1,женат / замужем,0,F,госслужащий,0,NaN,сделка с подержанным автомобилем
55,0,NaN,54,среднее,1,гражданский брак,1,F,пенсионер,1,NaN,сыграть свадьбу
...,...,...,...,...,...,...,...,...,...,...,...,...
21489,2,NaN,47,Среднее,1,женат / замужем,0,M,компаньон,0,NaN,сделка с автомобилем
21495,1,NaN,50,среднее,1,гражданский брак,1,F,сотрудник,0,NaN,свадьба
21497,0,NaN,48,ВЫСШЕЕ,0,женат / замужем,0,F,компаньон,0,NaN,строительство недвижимости
21502,1,NaN,42,среднее,1,женат / замужем,0,F,сотрудник,0,NaN,строительство жилой недвижимости


Можно заметить, что пропуски в данных не случайные, там где отсутствуют данные о проработанных днях, пропущены значения и для зарплаты

### Вывод

В данных 21525 строк, есть пропуски в двух столбцах, также необычные значения столбца "стаж работы". Имеющиеся пропуски не случайны

### Шаг 2. Предобработка данных

### Обработка пропусков

Взглянем на медианное значение стажа работы

In [4]:
df.days_employed.median()

-1203.369528770489

In [5]:
print(f"Наибольшее количество проработанных лет - {df.days_employed.max()/365}")

Наибольшее количество проработанных лет - 1100.6997273296713


Удалим столбец с отрицательными значениями дней и явными выбросами - 1100 лет вряд ли кто-то проработал

In [6]:
df = df.drop(columns='days_employed')

In [7]:
df.head()

,children,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу


Посмотрим на столбец с доходом

In [8]:
mean_income = df.total_income.mean()
mean_income

167422.30220817294

In [9]:
median_income = df.total_income.median()
median_income

145017.93753253992

In [10]:
max_income = df.total_income.max()
max_income

2265604.028722744

В целом, средняя зарплата не сильно превышает медианную, но заполним пропуски в зарплате медианным значением, поскольку есть выбросы, вроде максимального значения - 2 млн. 

In [11]:
df.total_income.fillna(median_income, inplace=True)

In [12]:
df.total_income.isnull().sum() #убедимся, что пропусков нет

0

In [13]:
df.head()

,children,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу


Посмотрим столбец children:

In [14]:
df.children.value_counts()

 0     14149
 1      4818
 2      2055
 3       330
 20       76
-1        47
 4        41
 5         9
Name: children, dtype: int64

Здесь можно отметить целых два артефакта, первое - наличие у людей -1 ребенка (возможно случайно добавился минус перед единицей), и, второе - 20 детей выглядит явным преувеличением, скорее всего это была ошибка при введении данных и у данного человека 2 ребенка. Исправим эти ошибки

In [15]:
df.loc[df.children == -1, "children"] = 1
df.loc[df.children == 20, "children"] = 2

Проверим

In [16]:
df.children.value_counts()

0    14149
1     4865
2     2131
3      330
4       41
5        9
Name: children, dtype: int64

Теперь это похоже на правду

### Взглянем на возраст заемщиков

In [17]:
df.dob_years.value_counts()

35    617
40    609
41    607
34    603
38    598
42    597
33    581
39    573
31    560
36    555
44    547
29    545
30    540
48    538
37    537
50    514
43    513
32    510
49    508
28    503
45    497
27    493
56    487
52    484
47    480
54    479
46    475
58    461
57    460
53    459
51    448
59    444
55    443
26    408
60    377
25    357
61    355
62    352
63    269
64    265
24    264
23    254
65    194
66    183
22    183
67    167
21    111
0     101
68     99
69     85
70     65
71     58
20     51
72     33
19     14
73      8
74      6
75      1
Name: dob_years, dtype: int64

Явными ошибками являются значения 0, вряд ли младенец приходил брать кредит. Скорее всего это очередная ошибка. Заменим эти значения на пустую строку, чтобы не потерять другие данные, важные для анализа

In [18]:
df.loc[df.dob_years == 0, "dob_years"] = " "

In [19]:
df[df.dob_years == " "].head()

,children,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
99,0,,Среднее,1,женат / замужем,0,F,пенсионер,0,71291.522491,автомобиль
149,0,,среднее,1,в разводе,3,F,сотрудник,0,70176.435951,операции с жильем
270,3,,среднее,1,женат / замужем,0,F,сотрудник,0,102166.458894,ремонт жилью
578,0,,среднее,1,женат / замужем,0,F,пенсионер,0,97620.687042,строительство собственной недвижимости
1040,0,,высшее,0,в разводе,3,F,компаньон,0,303994.134987,свой автомобиль


После замены всех 0 в возрасте на пустую строку, они не будут нас путать

In [20]:
df[df.gender == "XNA"]

,children,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
10701,0,24,неоконченное высшее,2,гражданский брак,1,XNA,компаньон,0,203905.157261,покупка недвижимости


Один интересный пол XNA, что бы это могло значить...

### Вывод

Мы удалили столбец с неправильными значениями стажа работы, поскольку нам эти данные не понадобятся в будущем для анализа.
Пропуски в столбце доходов заполнили медианным значением, поскольку есть выбросы.
Обнаружены значения -1 и 20 в столбце с количеством детей, предполагаем, что это означает соответственно 1 и 2 ребенка.
Возраст заемщиков варьируется от 21 до 75, а также есть нули, заменим их на пустую строку

### Замена типа данных

In [21]:
df.head(3)

,children,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья


In [22]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 11 columns):
children            21525 non-null int64
dob_years           21525 non-null object
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        21525 non-null float64
purpose             21525 non-null object
dtypes: float64(1), int64(4), object(6)
memory usage: 1.8+ MB


Приведем колонку total_income к типу int - к чему нам 6-ти значные копейки. Применим .astype(), так как значения у нас и так численные

In [23]:
df['total_income'] = df.total_income.astype(int)

In [24]:
df.head()

,children,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875,покупка жилья
1,1,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080,приобретение автомобиля
2,0,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885,покупка жилья
3,3,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628,дополнительное образование
4,0,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616,сыграть свадьбу


### Вывод

Здесь мы привели данные о доходе заемщиков к типу int методом astype(), так как данные у нас изначально были типа float.

### Обработка дубликатов

Посмотрим уникальные значения столбца education

In [25]:
df['education'].unique()

array(['высшее', 'среднее', 'Среднее', 'СРЕДНЕЕ', 'ВЫСШЕЕ',
       'неоконченное высшее', 'начальное', 'Высшее',
       'НЕОКОНЧЕННОЕ ВЫСШЕЕ', 'Неоконченное высшее', 'НАЧАЛЬНОЕ',
       'Начальное', 'Ученая степень', 'УЧЕНАЯ СТЕПЕНЬ', 'ученая степень'],
      dtype=object)

Заметим, что значения в столбце education написаны в разном регистре, исправим и проверим: 

In [26]:
df['education'] = df['education'].str.lower()

In [27]:
df['education'].unique()

array(['высшее', 'среднее', 'неоконченное высшее', 'начальное',
       'ученая степень'], dtype=object)

In [28]:
df.head()

,children,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875,покупка жилья
1,1,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080,приобретение автомобиля
2,0,33,среднее,1,женат / замужем,0,M,сотрудник,0,145885,покупка жилья
3,3,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628,дополнительное образование
4,0,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616,сыграть свадьбу


Найдем количество дупликатов

In [29]:
df.duplicated().sum()

71

In [30]:
df = df.drop_duplicates().reset_index(drop=True)

In [31]:
df.duplicated().sum() #убедимся

0

### Вывод

Произвели проверку дубликатов, с учетом разного регистра слов в столбце образования, удалось выявить 71 дубликат, избавимся от них и применим reset_index, чтобы обновить индексы

### Лемматизация

Посмотрим, для каких целей брали кредиты заемщики:

In [32]:
df.head()

,children,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875,покупка жилья
1,1,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080,приобретение автомобиля
2,0,33,среднее,1,женат / замужем,0,M,сотрудник,0,145885,покупка жилья
3,3,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628,дополнительное образование
4,0,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616,сыграть свадьбу


In [33]:
unique_purposes = df.purpose.unique()
unique_purposes

array(['покупка жилья', 'приобретение автомобиля',
       'дополнительное образование', 'сыграть свадьбу',
       'операции с жильем', 'образование', 'на проведение свадьбы',
       'покупка жилья для семьи', 'покупка недвижимости',
       'покупка коммерческой недвижимости', 'покупка жилой недвижимости',
       'строительство собственной недвижимости', 'недвижимость',
       'строительство недвижимости', 'на покупку подержанного автомобиля',
       'на покупку своего автомобиля',
       'операции с коммерческой недвижимостью',
       'строительство жилой недвижимости', 'жилье',
       'операции со своей недвижимостью', 'автомобили',
       'заняться образованием', 'сделка с подержанным автомобилем',
       'получение образования', 'автомобиль', 'свадьба',
       'получение дополнительного образования', 'покупка своего жилья',
       'операции с недвижимостью', 'получение высшего образования',
       'свой автомобиль', 'сделка с автомобилем',
       'профильное образование', 'высшее об

Можно выделить четыре основных цели, для которых люди брали кредиты:
    - На покупку/ремонт "жилья" или "недвижимости"
    - Проведение свадьбы
    - Покупка автомобиля
    - Получение образования

In [34]:
from pymystem3 import Mystem
m = Mystem()

In [35]:
def lemmatizer(text):
    stemmed_words = m.lemmatize(text)
    if ("жилье" in stemmed_words or "недвижимость" in stemmed_words):
        return 'недвижимость'
    if ("свадьба" in stemmed_words):
        return 'свадьба'   
    if ("автомобиль" in stemmed_words):
        return 'автомобиль'        
    if ("образование" in stemmed_words):
        return 'образование'

In [36]:
df['purpose'] = df['purpose'].apply(lemmatizer)

In [37]:
df.purpose.unique()

array(['недвижимость', 'автомобиль', 'образование', 'свадьба'],
      dtype=object)

In [38]:
df.head(10)

,children,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875,недвижимость
1,1,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080,автомобиль
2,0,33,среднее,1,женат / замужем,0,M,сотрудник,0,145885,недвижимость
3,3,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628,образование
4,0,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616,свадьба
5,0,27,высшее,0,гражданский брак,1,M,компаньон,0,255763,недвижимость
6,0,43,высшее,0,женат / замужем,0,F,компаньон,0,240525,недвижимость
7,0,50,среднее,1,женат / замужем,0,M,сотрудник,0,135823,образование
8,2,35,высшее,0,гражданский брак,1,F,сотрудник,0,95856,свадьба
9,0,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425,недвижимость


### Вывод

В данном пункте мы произвели лемматизацию и выделили 4 категории, на которые берут кредит заемщики - это недвижимость, автомобиль, образование, свадьба. Заменим значения столбца purpose на соответствующие категории.

### Категоризация данных

In [39]:
df.head()

,children,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875,недвижимость
1,1,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080,автомобиль
2,0,33,среднее,1,женат / замужем,0,M,сотрудник,0,145885,недвижимость
3,3,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628,образование
4,0,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616,свадьба


Посмотрим на квартили нашей выборки:

In [40]:
df.total_income.describe()

count    2.145400e+04
mean     1.652256e+05
std      9.802102e+04
min      2.066700e+04
25%      1.076230e+05
50%      1.450170e+05
75%      1.958132e+05
max      2.265604e+06
Name: total_income, dtype: float64

Нижний квартиль равен приблизительно 100 тыс. , а верхний около 200 тыс. Медиана приблизительно равна 150 тыс. Разобьем нашу выборку на четыре части:
    - тех, чей доход <100.000
    - доход от 100 до 150 тыс.
    - доход от 150 до 200 тыс.
    - и >200.000

In [41]:
def income_group(money):
    if money <= 100000:
        return '<100000'
    if money < 150000:
        return '<150000'
    if money < 200000:
        return '<200000'
    #if money > 350000:
        #return '>350000'
    return '>200000'

In [42]:
df['income_group'] = df['total_income'].apply(income_group)

In [43]:
df.head(10)

,children,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,income_group
0,1,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875,недвижимость,>200000
1,1,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080,автомобиль,<150000
2,0,33,среднее,1,женат / замужем,0,M,сотрудник,0,145885,недвижимость,<150000
3,3,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628,образование,>200000
4,0,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616,свадьба,<200000
5,0,27,высшее,0,гражданский брак,1,M,компаньон,0,255763,недвижимость,>200000
6,0,43,высшее,0,женат / замужем,0,F,компаньон,0,240525,недвижимость,>200000
7,0,50,среднее,1,женат / замужем,0,M,сотрудник,0,135823,образование,<150000
8,2,35,высшее,0,гражданский брак,1,F,сотрудник,0,95856,свадьба,<100000
9,0,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425,недвижимость,<150000


### Вывод

Добавили дополнительный столбец income_group, где есть отметка о том, к какой группе принадлежит заемщик. Это нам понадобится для дальнейшего анализа

### Шаг 3. Ответьте на вопросы

- Есть ли зависимость между наличием детей и возвратом кредита в срок?

In [44]:
part_of_people_with_debt = (df.groupby('children')['debt'].sum()) / (df.children.value_counts())

In [45]:
print('{:.1%} людей, у которых нет ребенка не возвращают кредит вовремя'.format(part_of_people_with_debt[0]))
print('{:.1%} людей, у которых один ребенок не возвращают кредит вовремя'.format(part_of_people_with_debt[1]))
print('{:.1%} людей, у которых два ребенка не возвращают кредит вовремя'.format(part_of_people_with_debt[2]))
print('{:.1%} людей, у которых три ребенка не возвращают кредит вовремя'.format(part_of_people_with_debt[3]))
print('{:.1%} людей, у которых четыре ребенка не возвращают кредит вовремя'.format(part_of_people_with_debt[4]))
print('{:.1%} людей, у которых пять детей не возвращают кредит вовремя'.format(part_of_people_with_debt[5]))

7.5% людей, у которых нет ребенка не возвращают кредит вовремя
9.2% людей, у которых один ребенок не возвращают кредит вовремя
9.5% людей, у которых два ребенка не возвращают кредит вовремя
8.2% людей, у которых три ребенка не возвращают кредит вовремя
9.8% людей, у которых четыре ребенка не возвращают кредит вовремя
0.0% людей, у которых пять детей не возвращают кредит вовремя


### Вывод

Для людей, у которых нет детей доля должников составляет 7,5 %, а для клиентов банка с детьми 9,2 %. Я могу объяснить это тем, что содержание детей - большая статья расходов, которая не всегда может быть предсказана заранее, и, при возникновении сложной ситуации, деньги идут на благо детей, а не на погашение кредита.

Гипотеза: чем больше детей, тем больше вероятность допустить просрочку.

- Есть ли зависимость между семейным положением и возвратом кредита в срок?

In [47]:
df.groupby("family_status")['debt'].sum().sort_values()

family_status
вдовец / вдова            63
в разводе                 85
Не женат / не замужем    274
гражданский брак         388
женат / замужем          931
Name: debt, dtype: int64

In [48]:
df.family_status.value_counts().sort_values()

вдовец / вдова             959
в разводе                 1195
Не женат / не замужем     2810
гражданский брак          4151
женат / замужем          12339
Name: family_status, dtype: int64

In [49]:
(df.groupby("family_status")['debt'].sum().sort_values()) / (df.family_status.value_counts().sort_values())*100

family_status
вдовец / вдова           6.569343
в разводе                7.112971
Не женат / не замужем    9.750890
гражданский брак         9.347145
женат / замужем          7.545182
dtype: float64

### Вывод

Зависимость безусловно есть: заемщики, которые не женаты/не замужем чаще не возвращают кредит в срок.
Реже всех задерживают с оплатой вдовы и вдовцы. Вероятно, это связано с тем, что клиент, лишившийся супруга или супруги, более опытен в финансовых вопросах - семейная жизнь научила распоряжаться деньгами грамотно. Это же касается людей, находящихся в разводе, у них есть опыт формирования финансового плана семьи. Люди не женатые и не замужние имеют меньше опыта в вопросах организации финансового благополучия, в связи с чем чаще допускают просрочки по оплате.

Гипотеза: Если человек был в браке (категории "вдова/вдовец", "в разводе") он реже допускает просрочки, а не женатые/не замужние чаще не выплачивают кредит вовремя.

- Есть ли зависимость между уровнем дохода и возвратом кредита в срок?

In [50]:
df.groupby('income_group')['debt'].sum()

income_group
<100000    354
<150000    661
<200000    368
>200000    358
Name: debt, dtype: int64

In [51]:
df.groupby('income_group')['income_group'].count()

income_group
<100000    4463
<150000    7807
<200000    4118
>200000    5066
Name: income_group, dtype: int64

In [52]:
(df.groupby('income_group')['debt'].sum()) / (df.groupby('income_group')['income_group'].count())*100

income_group
<100000    7.931884
<150000    8.466761
<200000    8.936377
>200000    7.066719
dtype: float64

### Вывод

Чаще не возвращают в срок кредит люди с зарплатой от 100 до 200 тыс. Те, у кого зарплата больше 200 тыс статистически чаще возвращают кредит вовремя. Я связываю это с бОльшей финансовой грамотностью у людей с большИм доходом, они умеют четко запланировать свои траты и выплачивают кредит вовремя.

Гипотеза: Клиент с большим доходом вероятнее выплатит кредит вовремя, а человек со средним доходом чаще допускает просрочки по оплате.

- Как разные цели кредита влияют на его возврат в срок?

In [53]:
df.groupby('purpose')['debt'].sum()

purpose
автомобиль      403
недвижимость    782
образование     370
свадьба         186
Name: debt, dtype: int64

In [54]:
df.groupby('purpose')['purpose'].count()

purpose
автомобиль       4306
недвижимость    10811
образование      4013
свадьба          2324
Name: purpose, dtype: int64

In [55]:
(df.groupby('purpose')['debt'].sum()) / (df.groupby('purpose')['purpose'].count())*100

purpose
автомобиль      9.359034
недвижимость    7.233373
образование     9.220035
свадьба         8.003442
dtype: float64

### Вывод

Люди с целями взятия кредита на операции с недвижимостью вероятнее всего возвращают кредит в срок, а вот для тех, кто берет кредит на автомобиль или образование эта доля меньше. Моя гипотеза заключается в том, что покупка недвижимости - это ответственный этап, требующий серьезного и скрупулезного подхода, в связи с чем выходит меньше просрочек. Для целей покупка автомобиля и получение образования сумма кредита изначально меньшая, что расширяет круг потенциальных клиентов для взятия этого кредита, и, как следствие, допускает менее ответственных людей. На проведение свадьбы же требуется меньше всего средств, что делает этот кредит более легким для погашения.

Гипотеза: человек, берущий кредит на операции с недвижимостью вероятнее вернет его вовремя, чем клиент, целями которого являются приобретение автомобиля, получение образования или свадьба.

### Шаг 4. Общий вывод

Нами были получены данные о 21525 заемщиках - их возраст, рабочий стаж, уровень образования, семейное положение, пол, кем работает, доход и цель взятия кредита, а также таргет значение - вернул или не вернул данный человек кредит в срок. 
В ходе изучения данных были обнаружены NaNы в столбцах общих рабочих дней и заработной платы, при этом столбец рабочие дни явны был испорчен - в нем были отрицательные значения и слишком большие (1000 лет стажа), поэтому дропнем этот столбец. Пропуски были в одних и тех же строках, значит пропуски не случайны. Заполним NaNы колонки з/п значениями медианной зарплаты. Перед обработкой дупликатов приведем данные столбца education к одному регистру - удалим все дубликаты и резетнем индекс.
Цели кредита можно представить как:
    1. Операции с недвижимостью
    2. Свадьба
    3. Покупка автомобиля
    4. Получение образования
Проведем лемматизацию целей, используя библиотеку pymystem3.

Данные о зарплате заемщиков разобьем на четыре подгруппы и запишем в дополнительный столбец:
    тех у кого зарплата < 100.000, 
    тех у кого з/п больше 100.000 и < 150.000
    и тех у кого доход > 150.000 и <200.000
    >200.000

Выводы:
1. Чаще всего возвращают кредит вовремя люди у которых нет детей, а реже всего те - у кого четыре ребенка.
2. Заемщики, которые не женаты/не замужем чаще не возвращают кредит в срок. Реже всех задерживают с оплатой вдовы и вдовцы.
3. Люди с большим доходом (>200 000) чаще всех возвращают кредит вовремя.
4. Те заемщики, которые берут кредит на недвижимость, чаще других возвращают кредит вовремя, а вот с целями на автомобиль, образование и свадьбу чаще задерживают оплату.